In [1]:
import random
import numpy as np
import numdifftools as nd
import jax.numpy as jnp
import sympy as sy
from jax import grad
from datetime import datetime
from math import sin, cos

from constants import *

Question 1

How many multiplications and additions do you need to perform a matrix multiplication between a (n, k) and (k, m) matrix? Explain

Answer:

matrix_multiplication = row * col \
total_multiplication = col_a* col_b * row_a \
total_addition = (col_a-1) * col_b * row_a

Question 2

Write Python code to multiply the above two matrices. Solve using list of lists and then use numpy. Compare the timing of both solutions. Which one is faster? Why?

In [2]:
def matrix_multiplication(mat_a, mat_b):
    try:
        start = datetime.now()
        if len(mat_a[0]) != len(mat_b):
            raise Exception("Matrix multiplication can't be performed")
        result_row = len(mat_a)
        result_col = len(mat_b[0])
        result = [[0 for col in range(result_col)] for row in range(result_row)]
        for row in range(len(mat_a)):
            for col in range(len(mat_b[0])):
                for k in range(len(mat_b)):
                    result[row][col] += mat_a[row][k] * mat_b[k][col]
        end = datetime.now()
        time_elapsed = end-start
        print(f"Total Time elapsed: {time_elapsed.total_seconds()}")
        return result
    except Exception as e:
        print(e)
        
def matrix_multiplication_numpy(mat_a, mat_b):
    start = datetime.now()
    result = np.matmul(mat_a,mat_b)
    end = datetime.now()
    time_elapsed = end-start
    print(f"Total Time elapsed using numpy: {time_elapsed.total_seconds()}")
    return result
    

In [3]:
row_a, col_a = SIZE_MATA
row_b, col_b = SIZE_MATB
mat_a = [[random.randint(1, 100) for col in range(col_a)] for row in range(row_a)]
mat_b = [[random.randint(1, 100) for col in range(col_b)] for row in range(row_b)]

result = matrix_multiplication(mat_a, mat_b)
result_numpy = matrix_multiplication_numpy(mat_a, mat_b)

Total Time elapsed: 3.9e-05
Total Time elapsed using numpy: 5.9e-05


Question 3:

Finding the highest element in a list requires one pass of the array. Finding the second highest element requires 2 passes of the the array. Using this method, what is the time complexity of finding the median of the array? Can you suggest a better method? Can you implement both these methods in Python and compare against numpy.median routine in terms of time?

In [4]:
def get_highest(arr):
    highest = -1e9
    for item in arr:
        if item>highest:
            highest = item
    return highest

def median_brute_force(arr):
    start = datetime.now()
    n = len(arr)
    if n%2==0:
        mid_point = int(n/2)
    else:
        mid_point = int((n+1)/2)
    median = None
    temp_arr = arr
    for i in range(mid_point):
        median = get_highest(temp_arr)
        temp_arr.remove(median)
    end = datetime.now()
    time_elapsed = end-start
    print(f"Total Time elapsed using bruteforce: {time_elapsed.total_seconds()}")
    return median

def median_optimise(arr):
    start = datetime.now()
    arr.sort()
    n = len(arr)
    if n%2==0:
        mid_point = int(n/2)
    else:
        mid_point = int((n+1)/2)
    end = datetime.now()
    time_elapsed = end-start
    print(f"Total Time elapsed: {time_elapsed.total_seconds()}")
    return arr[mid_point-1]

def median_numpy(arr):
    start = datetime.now()
    end = datetime.now()
    time_elapsed = end-start
    print(f"Total Time elapsed using numpy: {time_elapsed.total_seconds()}")
    return np.median(arr)

In [5]:
arr = [random.randint(1, 100) for n in range(SIZE_ARRAY)]
arr_opt = arr.copy()
arr_np = arr.copy()
median_bf = median_brute_force(arr=arr)
median_opt = median_optimise(arr_opt)
median_np = median_numpy(arr_np)
print(median_bf, median_opt, median_np)
"""For small n, all methods roughly have same time consumption, but for large n results are satisfactory
"""

Total Time elapsed using bruteforce: 18.659756
Total Time elapsed: 0.003653
Total Time elapsed using numpy: 1e-06
50 50 50.0


'For small n, all methods roughly have same time consumption, but for large n results are satisfactory\n'

Question 4:

What is the gradient of the following function with respect to x and y?

In [6]:
def f(x):
  return (x[0]**2)*x[1]+(x[1]**3)*sin(x[0])

def diff_wrt_x(x):
  return 2*x+cos(x)

def diff_wrt_x(y):
  return 1+3*(y**2)*sin(1)

# Finding gradient at (1, 1)
gradient = nd.Gradient(f)([1, 1])
print(diff_wrt_x(1),diff_wrt_x(1), gradient)

3.5244129544236893 3.5244129544236893 [2.54030231 3.52441295]


Question 5:

Use JAX to confirm the gradient evaluated by your method matches the analytical solution corresponding to a few random values of x and y

In [7]:
# for x=1 >>> f(1,y) = y+(y**3)*sin(1)

def grad_wrt_y(y):
  return y+(y**3)*sin(1)


# for y=1 >>> f(x,1) = x**2+sin(x)
def grad_wrt_x(x):
  return x**2+jnp.sin(x)

dfy = grad(grad_wrt_x)
dfx = grad(grad_wrt_y)
print(dfy(1.), dfx(1.))

2.5403023 3.5244129


Question 6

Use sympy to confirm that you obtain the same gradient analytically.

In [8]:
def f(x,y):
  return (x**2)*y+(y**3)*(sy.sin(x))

x = sy.Symbol('x')
y = sy.Symbol('y')


def diff_wrt_x(x,y):
  return sy.diff(f(x,y),x)

def diff_wrt_y(x,y):
  return sy.diff(f(x,y),y)

In [11]:
diff_wrt_x(x,y)

2*x*y + y**3*cos(x)

In [12]:
diff_wrt_y(x,y)

x**2 + 3*y**2*sin(x)

In [17]:
# Calculating gradient for (1, 1)
diff_wrt_x(x, y).subs(x,1).subs(y,1) ,diff_wrt_y(x, y).subs(x,1).subs(y,1)

(cos(1) + 2, 1 + 3*sin(1))

Question 7:

Create a Python nested dictionary to represent hierarchical information. We want to store record of students and their marks. Something like:

In [4]:
database = {}
for year in ACAD_YEAR:
    database[year] = {}
    for branch in BRANCH:
        student_info = STUDENT
        student_info['roll_number'] = random.randint(1, 100)
        student_info['name'] = ""
        student_info['marks'] = MARKS
        database[year][branch] = student_info
print(database['2022'])

{'Electrical': {'roll_number': 90, 'name': '', 'marks': {'Math': 89, 'English': 34, 'Sanskrit': 76}}, 'Chemical': {'roll_number': 90, 'name': '', 'marks': {'Math': 89, 'English': 34, 'Sanskrit': 76}}, 'Materials': {'roll_number': 90, 'name': '', 'marks': {'Math': 89, 'English': 34, 'Sanskrit': 76}}}
